<a href="https://colab.research.google.com/github/HenryLiu714/Sentiment-Analysis-Model/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing and Importing Dependencies**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

In [5]:
# Ensure dependencies are installed

dependencies = [
    "numpy",
    "pandas",
    "matplotlib",
    "tweetnlp"
]

for dependency in dependencies:
  os.system(f"pip3 install {dependency}")

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tweetnlp

In [7]:
os.chdir("drive/MyDrive/sentiment_bot")

**Sentiment Analysis Model Loading**

In [8]:
model = tweetnlp.load_model("sentiment")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def get_sentiment(s: str) -> int:
  sentiment = model.sentiment(s)['label']

  if sentiment == 'positive':
    return 1
  elif sentiment == 'negative':
    return -1
  else:
    return 0

In [10]:
tweets = pd.read_csv('stock.csv', on_bad_lines='skip')

In [11]:
tweets = tweets.dropna()
tweets = tweets[tweets['category']=='stock_images']
tweets = tweets.drop(columns=['tweet_url', 'tweet_type', 'price_of_ticker', 'change_of_ticker', 'category'])

In [12]:
def tickers_to_array(s:str)->list[str]:
  s = s[1:-1].split(', ')
  tickers = []
  for ticker in s:
    tickers.append(ticker[2:-1])
  return tickers

tweets['tickers_mentioned'] = tweets['tickers_mentioned'].transform(lambda x: tickers_to_array(x))
tweets['timestamp'] = tweets['timestamp'].transform(lambda x: pd.Timestamp(x[:10]))

In [13]:
class Trade:
  def __init__(self, ticker: str, dir: bool, strength: int):
    self.ticker = ticker
    self.dir = dir
    self.strength = strength

In [14]:
def get_best_trade(date: pd.Timestamp) -> Trade:
  ticker_data = tweets[tweets['timestamp'] == date]

  tickers = {}

  for index, row in ticker_data.iterrows():
    sentiment = get_sentiment(row['tweet_text'])

    for ticker in row['tickers_mentioned']:
      if ticker not in tickers:
        tickers[ticker] = sentiment
      else:
        tickers[ticker] += sentiment

  best_ticker = max(tickers, key=tickers.get)
  worst_ticker = min(tickers, key=tickers.get)

  if abs(tickers[best_ticker]) > abs(tickers[worst_ticker]):
    return Trade(best_ticker, True, abs(tickers[best_ticker]))
  else:
    return Trade(worst_ticker, False, abs(tickers[worst_ticker]))


In [70]:
import yfinance as yf
import json

cache = {}
with open('cache.json', 'r') as f:
  cache = json.load(f)

In [54]:
class TradeResult:
  def __init__(self, ticker: str, date: str, start: float, end: float):
    self.ticker = ticker
    self.date = date
    self.start = start
    self.end = end
    self.pnl = end - start
  def show_trade(self):
    print(f"Ticker: {self.ticker}")
    print(f"Start: {self.start}")
    print(f"End: {self.end}")
    print(f"Pnl: {self.pnl}")
    print("----------------------\n")

In [86]:
def backtest(starting_balance: int = 100000, port_split: int = 10, commission: float = 0.002, end_date: str = "2023-11-20") -> int:
  cost_per_trade = starting_balance / port_split
  balance = starting_balance
  curr_balance = balance

  #equities we hold :3
  equities = []
  trades = []

  for date in tweets['timestamp'].unique():
    print(f"Date: {date}, Best Trade: {get_best_trade(date).ticker} ")

    trade = get_best_trade(date)
    if trade.dir:
      print("BUYING")
      try:
        # Caching because i dont wanna wait
        if date.strftime('%Y-%m-%d') in cache and cache[date.strftime('%Y-%m-%d')][0] == trade.ticker:
          price = cache[date.strftime('%Y-%m-%d')][1]
        else:
          data = yf.download(trade.ticker, start=date.strftime('%Y-%m-%d'), end=(date + pd.Timedelta(days = 4)).strftime('%Y-%m-%d'))
          price = data.iloc[0]['Adj Close']
          cache[date.strftime('%Y-%m-%d')] = [trade.ticker, price]

        if balance > 0:
          #price = rice # henry slays # henry is a cat # inf/tj men >> #henry has emotions shooketh # henry si more f than t based on prior memories # henry are we done yet :O
          print(f"Current balance: {balance}")
          print(f"Cost of trade: {cost_per_trade}")
          equities.append([trade.ticker, cost_per_trade/price])
          balance -= cost_per_trade

          # Add to trade_results
          trades.append(TradeResult(trade.ticker, date.strftime('%Y-%m-%d'), cost_per_trade, 0))
        else:
          first_trade = equities.pop(0)
          first_price = yf.download(first_trade[0], start=date.strftime('%Y-%m-%d'), end=(date + pd.Timedelta(days = 4)).strftime('%Y-%m-%d')).iloc[0]['Adj Close']
          cash_on_hand = first_price * first_trade[1] * (1 - commission) # After selling first stock :3
          print(f"Cost of trade: {cash_on_hand}")

          equities.append([trade.ticker, cash_on_hand/price])

          # Add to trade results
          trades[-port_split].end = cash_on_hand
          trades.append(TradeResult(trade.ticker, date.strftime('%Y-%m-%d'), cash_on_hand, 0))
      except:
        print(f"Error: Symbol {trade.ticker} not found")


    # Getting all the money (hayun's favorite part >:3)
  for equity in equities:
    data = yf.download(equity[0], start=end_date, end=(pd.Timestamp(end_date) + pd.Timedelta(days = 4)).strftime('%Y-%m-%d'))
    price = data.iloc[0]['Adj Close']

    balance += equity[1] * price

  with open('trades.txt', 'w') as f:
    for trade in trades:
      f.write(f"{trade.date} : {trade.ticker} : From: {trade.start} To: {trade.end} PnL: {trade.pnl}\n")

  return balance, trades


In [77]:
def document_trades(trades: list[TradeResult]):
  with open('trades.txt', 'w') as f:
    for trade in trades:
      f.write(f"{trade.date} : {trade.ticker} : From: {trade.start} To: {trade.end} PnL: {trade.pnl}\n")

In [ ]:
returns, trades_made = backtest()

Date: 2021-12-27 00:00:00, Best Trade: COIN 
BUYING
Current balance: 100000
Cost of trade: 10000.0
Date: 2022-01-02 00:00:00, Best Trade: AMD 
BUYING
Current balance: 90000.0
Cost of trade: 10000.0
Date: 2022-01-03 00:00:00, Best Trade: TSLA 
BUYING
Current balance: 80000.0
Cost of trade: 10000.0
Date: 2022-01-04 00:00:00, Best Trade: F 
BUYING
Current balance: 70000.0
Cost of trade: 10000.0
Date: 2022-01-05 00:00:00, Best Trade: CRM 
Date: 2022-01-06 00:00:00, Best Trade: AMC 
Date: 2022-05-19 00:00:00, Best Trade: ^SPX 
Date: 2022-05-20 00:00:00, Best Trade: ES=F 
Date: 2022-05-21 00:00:00, Best Trade: ^SPX 
BUYING
Current balance: 60000.0
Cost of trade: 10000.0
Date: 2022-05-22 00:00:00, Best Trade: AAPL 
BUYING
Current balance: 50000.0
Cost of trade: 10000.0
Date: 2022-05-23 00:00:00, Best Trade: SBUX 
BUYING
Current balance: 40000.0
Cost of trade: 10000.0
Date: 2022-05-24 00:00:00, Best Trade: TWTR 
Date: 2022-05-25 00:00:00, Best Trade: TSLA 
BUYING
Current balance: 30000.0
Cost 

[*********************100%%**********************]  1 of 1 completed

BUYING
Cost of trade: 2781.0254944338926


Date: 2022-05-31 00:00:00, Best Trade: RVP 


[*********************100%%**********************]  1 of 1 completed

...

BUYING
Cost of trade: 9017.814692113956


Date: 2022-08-08 00:00:00, Best Trade: SPX 
Date: 2022-08-09 00:00:00, Best Trade: ES=F 
Date: 2022-08-10 00:00:00, Best Trade: AAPL 


[*********************100%%**********************]  1 of 1 completed

BUYING
Cost of trade: 11568.145777839349


Date: 2022-08-11 00:00:00, Best Trade: DKNG 
Date: 2022-08-12 00:00:00, Best Trade: DNMR 


In [ ]:
out_file = open("cache.json", "w")
out_file.write(json.dumps(cache))
out_file.close()